In [1]:
import numpy as np
import pickle
import torch

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from utils import SimpleProblem

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.set_default_dtype(torch.float64)

import gurobipy as gp
from gurobipy import Model, GRB, QuadExpr, LinExpr

from dcopf_utils import build_gurobi_model, build_osqp_model
import time
from dcopf_utils import DcopfProblem

In [48]:
from model_utils import NNSolver_eq_proj

filepath = "/home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf200_data"
with open(filepath, 'rb') as f:
    data = pickle.load(f)

data = DcopfProblem(data, valid_frac=0.0833, test_frac=0.001)

arg_path = "/home/jxxiong/A-xjx/DC3/results/DcopfProblem-483-490-446-10000/method_eq_proj/0254a871350602cb2ca6f1b6e51e32f7ce2494ab/1713423666-7018242/args.dict"
args = pickle.load(open(arg_path, 'rb'))

model = NNSolver_eq_proj(data, args)

model_path = "/home/jxxiong/A-xjx/DC3/results/DcopfProblem-483-490-446-10000/method_eq_proj/0254a871350602cb2ca6f1b6e51e32f7ce2494ab/1713423666-7018242/solver_net.dict"
model.load_state_dict(torch.load(model_path))

model.eval()

NNSolver_eq_proj(
  (net): Sequential(
    (0): Linear(in_features=446, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=1024, out_features=1024, bias=True)
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GELU(approximate='none')
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=1024, out_features=483, bias=True)
  )
)

In [49]:
DEVICE = 'cpu'
model.to(DEVICE)
for attr in dir(data):
    var = getattr(data, attr)
    if not callable(var) and not attr.startswith("__") and torch.is_tensor(var):
        try:
            setattr(data, attr, var.to(DEVICE))
        except AttributeError:
            pass

In [50]:
A = data.A
G = data.G
h = data.h
Q = data.Q
p = data.p
Lb = data.Lb
Ub = data.Ub

A_np = data.A_np
G_np = data.G_np
h_np = data.h_np
Q_np = data.Q_np
p_np = data.p_np
Lb_np = data.Lb.cpu().numpy()
Ub_np = data.Ub.cpu().numpy()

X = data.X
X_np = data.X_np

num_var = A.shape[1]
num_eq = A.shape[0]
num_ineq = G.shape[0]

In [51]:
p.max()

tensor(2319.4000)

In [52]:
idx = 1

In [53]:
# full_model = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
# start_time = time.time()
# r = full_model.solve()
# print("OSQP time: ", time.time() - start_time)
# print(r.info.obj_val)
# x_full = r.x

In [54]:
# Lb_np_reduced = Lb_np.copy()
# Lb_np_reduced[inactive_lower_bounds] = -np.inf
# Ub_np_reduced = Ub_np.copy()
# Ub_np_reduced[inactive_upper_bounds] = np.inf

# reduced_model = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np_reduced, Ub_np_reduced)
# start_time = time.time()
# r = reduced_model.solve()
# print("OSQP reduced time: ", time.time() - start_time)
# print(r.info.obj_val)
# x_reduced = r.x

In [55]:
model_presolve = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
%time model_presolve = model_presolve.presolve()
# reduced_model.write("reduced_model.lp")

Set parameter FeasibilityTol to value 0.0001
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Presolve removed 547 rows and 220 columns
CPU times: user 3.44 ms, sys: 442 µs, total: 3.88 ms
Wall time: 3.73 ms


In [56]:
full_model = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
full_model.setParam('OutputFlag', 1)
full_model.setParam('Presolve', -1)
full_model.optimize()
optimal_val_full = full_model.objVal
time_full = full_model.Runtime
optimal_x_full = np.array([v.x for v in full_model.getVars()])
duals_full = np.array([c.Pi for c in full_model.getConstrs()])
duals_ineq_target = duals_full[-G.shape[0]:]
duals_eq_target = duals_full[:A.shape[0]]
active_ineq_target = np.where(np.abs(G_np@optimal_x_full - h_np) <= 1e-4)[0]
active_ineq_target.tolist()

active_lower_bounds = np.where(optimal_x_full - Lb_np <= 1e-4)[0]
active_upper_bounds = np.where(Ub_np - optimal_x_full <= 1e-4)[0]
print(active_ineq_target)
print(active_lower_bounds)
print(active_upper_bounds)

Set parameter FeasibilityTol to value 0.0001
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 936 rows, 483 columns and 2244 nonzeros
Model fingerprint: 0x27ada556
Model has 31 quadratic objective terms
Coefficient statistics:
  Matrix range     [4e-01, 3e+02]
  Objective range  [7e+02, 2e+03]
  QObjective range [4e+01, 4e+01]
  Bounds range     [7e-03, 7e+00]
  RHS range        [3e-03, 8e-01]
Presolve removed 771 rows and 337 columns
Presolve time: 0.00s
Presolved: 165 rows, 196 columns, 723 nonzeros
Presolved model has 31 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.555e+03
 Factor NZ  : 3.959e+03
 Factor Ops : 1.055e+05 (less than 1 second per iteration)
 Threads    : 1

                  Object

In [57]:
inactive_lower_bounds = np.where(optimal_x_full - Lb_np > 1e-4)[0]
inactive_upper_bounds = np.where(Ub_np - optimal_x_full > 1e-4)[0]

In [58]:
# calculate the parameter values
# optimal_val_full = 955340
# num_ineq = 11424 + 7024*2
opt_gap = 1e-2
fac = max(1, max(np.abs(Q_np).max(), np.abs(p_np).max()))
t = min(fac * (num_ineq + num_var * 2) / opt_gap / abs(optimal_val_full), 1000)
dual_threshold = 1e-3
ineq_threshold = 1 / dual_threshold / (int(t))

print(int(t), ineq_threshold)

1000 1.0


In [59]:
(t / 5) ** (1/20)

1.3033213205630634

# remove part of the inequality constraints

In [60]:
# start_time = time.time()
# Yhat = model(X[idx].unsqueeze(0).to('cuda'))
# Ystar = data.projection(X[idx].unsqueeze(0).to('cuda'), Yhat)
# end_time = time.time()


In [61]:
start_time = time.time()
Yhat = model(X[idx].unsqueeze(0))
Ystar = data.projection(X[idx].unsqueeze(0), Yhat)
end_time = time.time()
inference_time = end_time - start_time
print("Inference time: ", inference_time)
Ystar = Ystar.detach().numpy().flatten()
ineq_sorted_index = np.argsort(-G_np@Ystar + h_np)
ineq_sorted = np.sort(-G_np@Ystar + h_np)
print(ineq_sorted)
print(ineq_sorted_index)
inactive_ineq_index = ineq_sorted_index[ineq_sorted > ineq_threshold]
print(inactive_ineq_index)

lb_sorted_index = np.argsort(Ystar - Lb_np)
lb_sorted = np.sort(Ystar - Lb_np)
inactive_lower_bounds = lb_sorted_index[lb_sorted > ineq_threshold]

ub_sorted_index = np.argsort(Ub_np - Ystar)
ub_sorted = np.sort(Ub_np - Ystar)
inactive_upper_bounds = ub_sorted_index[ub_sorted > ineq_threshold]

Inference time:  0.002658843994140625
[0.42959524 0.43151486 0.43568924 0.43980979 0.46410336 0.46590803
 0.47154478 0.47455982 0.47602509 0.47702366 0.47809128 0.4796043
 0.48091734 0.48509701 0.48536566 0.48577572 0.48705723 0.48827908
 0.48846527 0.48851821 0.49021851 0.49138945 0.49398164 0.4941731
 0.4946991  0.494708   0.49519623 0.49525998 0.49528639 0.49611251
 0.49664667 0.49674434 0.49697256 0.49711584 0.49716308 0.4971704
 0.49787072 0.49829057 0.49865567 0.49873758 0.49973804 0.49995629
 0.50040442 0.50085796 0.50162319 0.50172857 0.50238465 0.50241781
 0.50251346 0.5027364  0.50276038 0.50295821 0.50302901 0.50307926
 0.50324109 0.50339219 0.5035378  0.50360685 0.50362843 0.50434002
 0.50499459 0.50558741 0.50575608 0.50583738 0.50600556 0.50609211
 0.50619928 0.50672364 0.50674597 0.50703202 0.50742753 0.50748278
 0.5075297  0.50768329 0.5077036  0.50777386 0.50806425 0.50815667
 0.5083113  0.50843354 0.50856871 0.5086272  0.50883317 0.50889517
 0.50922663 0.50932499 0.50

In [62]:
set(inactive_ineq_index).intersection(set(active_ineq_target))

set()

In [63]:
remaining_ineq_index = np.setdiff1d(np.arange(num_ineq), inactive_ineq_index)
G_remaining = G[remaining_ineq_index]
h_remaining = h[remaining_ineq_index]

lb_remaining = Lb_np.copy()
ub_remaining = Ub_np.copy()
lb_remaining[inactive_lower_bounds] = -np.inf
ub_remaining[inactive_upper_bounds] = np.inf

In [64]:
model_reduced = build_gurobi_model(Q, p, A, X_np[idx], G_remaining, h_remaining, lb_remaining, ub_remaining)
model_reduced.setParam('OutputFlag', 1)
model_reduced.setParam('Presolve', -1)
# initialize the primal variables in model_hat2
# for i, v in enumerate(model_hat2.getVars()):
#     v.setAttr('Start', Ystar[i])
model_reduced.optimize()
optimal_val = model_reduced.objVal
time_reduced = model_reduced.Runtime + inference_time
optimal_x = np.array([v.x for v in model_reduced.getVars()])
print("Value: full={}, reduced={}, diff={}".format(optimal_val_full, optimal_val, optimal_val_full - optimal_val))
print("Time: full={}, reduced={}, diff={}".format(time_full, time_reduced, time_full - time_reduced))

Set parameter FeasibilityTol to value 0.0001
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 936 rows, 483 columns and 2244 nonzeros
Model fingerprint: 0x0581cbef
Model has 31 quadratic objective terms
Coefficient statistics:
  Matrix range     [4e-01, 3e+02]
  Objective range  [7e+02, 2e+03]
  QObjective range [4e+01, 4e+01]
  Bounds range     [7e-03, 2e+00]
  RHS range        [3e-03, 8e-01]
Presolve removed 758 rows and 392 columns
Presolve time: 0.00s
Presolved: 178 rows, 209 columns, 755 nonzeros
Presolved model has 31 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 1.953e+03
 Factor NZ  : 4.026e+03
 Factor Ops : 1.026e+05 (less than 1 second per iteration)
 Threads    : 1

        

In [65]:
full_model.Runtime, model_reduced.Runtime

(0.010710000991821289, 0.009763002395629883)

# osqp warm start

In [43]:
num_trial = 10

total_time = 0.0
for _ in range(num_trial):
    solver = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
    # solver1.warm_start(x=Ystar)
    start_time = time.time()
    results = solver.solve()
    end_time = time.time()
    total_time += (end_time - start_time)
print(total_time / num_trial)
print(results.info.obj_val)


0.015122532844543457
13375.88303514959


In [30]:
total_time_ws = 0.0
for _ in range(num_trial):
    solver_ws = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np, Lb_np, Ub_np)
    solver_ws.warm_start(x=Ystar)
    start_time = time.time()
    result_ws = solver_ws.solve()
    end_time = time.time()
    total_time_ws += (end_time - start_time)
print(total_time_ws / num_trial + inference_time)
print(result_ws.info.obj_val)

0.016102910041809082
13375.848118813015


In [31]:
total_time_reduced = 0.0
for _ in range(num_trial):
    solver_reduced = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining, lb_remaining, ub_remaining)
    start_time = time.time()
    results_reduced = solver_reduced.solve()
    end_time = time.time()
    total_time_reduced = end_time - start_time
print(total_time_reduced / num_trial + inference_time)
print(results_reduced.info.obj_val)

0.0038568258285522463
13375.833319200805


In [32]:
total_time_reduced_ws = 0.0
for _ in range(num_trial):
    solver_reduced_ws = build_osqp_model(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining, lb_remaining, ub_remaining)
    solver_reduced_ws.warm_start(x=Ystar)
    start_time = time.time()
    results_reduced_ws = solver_reduced_ws.solve()
    end_time = time.time()
    total_time_reduced_ws = end_time - start_time
print(total_time_reduced_ws / num_trial + inference_time)
print(results_reduced_ws.info.obj_val)

0.003991389274597168
13375.773211036943


In [33]:
x = results_reduced.x
data.ineq_dist(X_np[idx], torch.tensor(x).unsqueeze(0)).max()

tensor(5.0846e-09)

# check

In [23]:
error = []
for i in range(9900, 10000):
    full_model = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
    full_model.optimize()
    optimal_x_full = np.array([v.x for v in full_model.getVars()])
    active_ineq_target = np.where(np.abs(G_np@optimal_x_full - h_np) <= 1e-4)[0]
    active_ineq_target.tolist()

    Yhat = model(X[idx].unsqueeze(0))
    Ystar = data.projection(X[idx].unsqueeze(0), Yhat)
    Ystar = Ystar.detach().numpy().flatten()
    ineq_sorted_index = np.argsort(-G_np@Ystar + h_np)
    ineq_sorted = np.sort(-G_np@Ystar + h_np)
    inactive_ineq_index = ineq_sorted_index[ineq_sorted > ineq_threshold]

    error.append(len(set(inactive_ineq_index).intersection(set(active_ineq_target))))
print(max(error))



0


In [24]:
error

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]